## Import Libraries

In [9]:
# %matplotlib ipympl
# %matplotlib inline
%matplotlib wx

In [10]:
import matplotlib.pyplot as plt
plt.ion()

In [11]:
from pydgilib_extra import *
from atprogram import atprogram

In [12]:
from os import path, pardir

## Compile and program project

In [13]:
project_path = [path.curdir, "BitCounting-S"]
project_path

['.', 'BitCounting-S']

In [14]:
atprogram.atprogram("./BitCounting-S", device_name = "ATSAML21J18B", verbose=3)

make: Nothing to be done for 'all'.

[DEBUG] Starting execution of "chiperase"
[DEBUG] Starting process 'C:\Program Files (x86)\Atmel\Studio\7.0\atbackend\atbackend.exe'
[DEBUG] Connecting to TCP:127.0.0.1:52348
[INFO] Connected to edbg, fw version: 3.25
[INFO] Firmware check OK
[DEBUG] Command "chiperase" finished with return code 0
[DEBUG] Starting execution of "program"
[DEBUG] Memory segment base written at 0x00000000. Size = 0x000007d4.
[DEBUG] Command "program" finished with return code 0
[DEBUG] Starting execution of "info"
[ERROR] Error in TCF lockbit format. Missed key Could not get Module LOCKBIT for ATSAML21J18B. (TCF Error code: 131120)
[DEBUG] Command "info" finished with return code 0
[DEBUG] Exit successfully.
Firmware check OK
Chiperase completed successfully
Programming completed successfully.
Tool edbg has firmware version: 03.25
Target voltage: 3.31 V

Device information:

Name:       ATSAML21J18B  
JtagId:     N/A           
Revision:   C             
Chip ID:    0x

0

In [15]:
atprogram.get_project_size("./BitCounting-S", device_name = "ATSAML21J18B" , verbose=2)

{'text': 2004, 'data': 0, 'bss': 17128, 'dec': 19132, 'hex': 19132, 'filename': 'BitCounting-S.elf'}


{'text': 2004,
 'data': 0,
 'bss': 17128,
 'dec': 19132,
 'hex': 19132,
 'filename': 'BitCounting-S.elf'}

## Data Logging

In [31]:
live_plot = True

Create a figure for the plot.

In [32]:
if live_plot:
    fig = plt.figure(figsize=(10, 6))
    fig.show()

Create the configuration dictionary for `DGILibExtra`.

In [33]:
config_dict = {
    "loggers": [LOGGER_OBJECT, LOGGER_CSV],
    "file_name_base": "experiment_bit_counting"
}
config_dict_plot = {
    "loggers": [LOGGER_OBJECT, LOGGER_PLOT, LOGGER_CSV],
    "plot_pins": [False, False, True, False],
    "plot_pins_method": "line",
    "plot_xmax": 0.6,
    "window_title": "Experiment BitCounting",
}

Stop criteria to pass to the logger:

In [34]:
def stop_fn(logger_data):
    return len(logger_data.gpio) and all(logger_data.gpio.values[-1])

Stop criteria to pass to the parser:

In [35]:
def stop_function(pin_values):
    return all(pin_values)

In [36]:
repetitions = 10

Perform the measurement.

In [37]:
data = []
cd = config_dict.copy()
if live_plot:
    fig.clf()
    for ax in fig.get_axes():
        ax.cla()
    
    cd.update(config_dict_plot)
    cd["fig"] = fig
        
charges = []
times = []

with DGILibExtra(**cd) as dgilib:
    for _ in range(repetitions):
        dgilib.device_reset()
        dgilib.logger.log(2, stop_fn)
        charge, time = power_and_time_per_pulse(dgilib.data, 2, stop_function=stop_function)
        charges += charge
        times += time
        if len(charge) != 1 or len(time) != 1:
            print(f"Parsing of measurement data failed, charge:{charge}, time:{time}")
            break
        dgilib.empty_data()
    
    dgilib.logger.plotobj.ax.set_title(f"Average of {repetitions} samples: charge: {sum(charges)/len(charges)*1e3:.06} mC, time: {sum(times)/len(times):.06} s")

In [38]:
import json

config = {}
config["name"] = "Bit Counting 2048"
config["project_paths"] = [project_path]
config["config_dict"] = config_dict
config["config_dict_plot"] = config_dict_plot
config["analysis"] = {"pins":{2: ["Bit Counting"]}, 
                      "result_types": ["Charge", "Time"], 
                      "section_types": {"init": [], 
                                        "loop": ["Bit Counting"],
                                        "exit": []}}

with open("repeated_experiment.json", 'w') as config_file:  
    json.dump(config, config_file, indent=4)

## Generate test data

In [39]:
NUM_UINTS = 2048
bitcount = "".join([bin(n) for n in range(NUM_UINTS)]).count('1')
print(bitcount)

11264


## Analysis

In [40]:
import pandas as pd

In [41]:
df = pd.DataFrame({'times': times, 'charges':charges})

In [42]:
df

,times,charges
0,0.016433,0.000007
1,0.016367,0.000007
2,0.016435,0.000007
3,0.016367,0.000007
4,0.016433,0.000007
5,0.016369,0.000007
6,0.016433,0.000007
7,0.016433,0.000007
8,0.016433,0.000007
9,0.016367,0.000007


In [43]:
df.describe()

,times,charges
count,10.000000,1.000000e+01
mean,0.016407,6.565583e-06
std,0.000034,1.713389e-08
min,0.016367,6.541591e-06
25%,0.016368,6.548695e-06
50%,0.016433,6.568209e-06
75%,0.016433,6.580452e-06
max,0.016435,6.584458e-06


In [44]:
sum(charges)/repetitions

6.565582935476292e-06

In [45]:
print("%.0100f" % (sum(charges)/repetitions))

0.0000065655829354762916143023118720201125597668578848242759704589843750000000000000000000000000000000
